## Importamos librerias a utilizar

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime
from Conexion_DB import ConexionDB

## Variables de entorno a utilizar

In [118]:
load_dotenv()

url_csv= os.getenv('URL')
server= os.getenv('SQL_SERVER')
db=os.getenv('SQL_DATABASE')
user=os.getenv('SQL_USER')
password=os.getenv('SQL_PASSWORD')
usuario = os.getlogin()

## Metodo para obtencion archivo CSV

In [119]:
def obtener_archivo(url,ruta):
    response= requests.get(url)
    if response.status_code == 200:
        with open(ruta,"wb") as documento:
            documento.write(response.content)
        print(f'Archivo descargado con exito y guardado en: {ruta}')
        return None
    else:
        print(f'Error al descargar el archivo, codigo del error : {response.status_code}')
        return None


## Guardado del archivo csv origen

In [120]:
fecha_hoy= datetime.now().strftime("%y-%m-%d")
anio= datetime.now().year
mes= datetime.now().month

nombre_csv= f"Nuevos_registros_{fecha_hoy}.csv"

ruta= os.path.join(os.getcwd(), f'Documentos\\{anio}\\{mes}')
ruta_final= os.path.join(ruta,nombre_csv)

os.makedirs(ruta,exist_ok=True)
obtener_archivo(url_csv,ruta_final)

Archivo descargado con exito y guardado en: f:\Proyectos\ChallengePI\Documentos\2025\7\Nuevos_registros_25-07-02.csv


## Metodo para limpieza de datos 

In [121]:
def limpieza_datos(ruta):
    df_csv=pd.read_csv(ruta)
    df_csv['FECHA_COPIA'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    df_final= df_csv.drop_duplicates(subset=['ID','MUESTRA','RESULTADO'], keep='last')
    registros=df_final.shape[0]
    return df_final,registros

In [122]:
df,Registros_Archivo= limpieza_datos(ruta_final)

## Conexion a DB y Carga de datos

In [123]:
conexion = ConexionDB()
conn = conexion.conectar()

In [124]:
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM [Testing_ETL].[dbo].[Unificado_test];")
total_registros= cursor.fetchone()[0]

In [125]:
cursor.execute("""
IF NOT EXISTS (SELECT * FROM tempdb.sys.tables WHERE name = 'Accion')
BEGIN
    CREATE TABLE #Accion (
        Tipo NVARCHAR(10),
        ID NVARCHAR(50),
        MUESTRA NVARCHAR(50),
        RESULTADO NVARCHAR(50)
    );
END;
""")

In [ ]:
fecha_inicio_Proceso= datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
for index, row in df.iterrows():
    set_values = ', '.join([f"{col} = '{row[col]}'" if isinstance(row[col], str) else f"{col} = {row[col]}" for col in df.columns])
    
    query = f"""
    MERGE INTO [Testing_ETL].[dbo].[Unificado_Test] AS target
    USING (SELECT '{row['ID']}' AS ID, '{row['MUESTRA']}' AS MUESTRA, '{row['RESULTADO']}' AS RESULTADO) AS source
    ON target.ID = source.ID AND target.MUESTRA = source.MUESTRA AND target.RESULTADO = source.RESULTADO
    WHEN MATCHED THEN
        UPDATE SET {set_values}
    WHEN NOT MATCHED THEN
        INSERT ({', '.join(df.columns)}) VALUES ({', '.join([f"'{x}'" if isinstance(x, str) else str(x) for x in row])})
    OUTPUT $action AS Tipo, inserted.ID, inserted.MUESTRA, inserted.RESULTADO INTO #Accion;
    """

    cursor.execute(query)
fecha_Fin_Proceso= datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

In [127]:
cursor.execute("SELECT COUNT(*) FROM #Accion WHERE Tipo = 'INSERT';")
insertados = cursor.fetchone()[0]


cursor.execute("SELECT COUNT(*) FROM #Accion WHERE Tipo = 'UPDATE';")
Actualizados = cursor.fetchone()[0]


In [128]:
querylog = f"""
INSERT INTO [dbo].[logs_Unificado_Test] (Archivo, Registro_Tabla, Registros_Archivo,Insertados,Actualizados,Fecha_Inicio_Proceso,Fecha_Fin_Proceso,usuario)
VALUES (?,?,?,?,?,?,?,?)
"""
cursor.execute(querylog, nombre_csv, total_registros, Registros_Archivo,insertados,Actualizados,fecha_inicio_Proceso,fecha_Fin_Proceso,usuario)

In [129]:
conn.commit()
cursor.close()
conexion.cerrar()